# Assigment: PyTorch Model Zoo

The [PyTorch "Model Zoo"](https://pytorch.org/vision/stable/models.html) provides a large number of pre-trained CNN models and vision [data sets](https://pytorch.org/vision/stable/datasets.html)...

In [1]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time
import copy

In [2]:
#transform input data (image) to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#set batch size
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Assignment 1:
Load a "*ResNet18*" from the torchvision model zoo and train it for 10 epochs

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.resnet18(weights=None)

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = trainloader
            else:
                model.eval()
                dataloader = testloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10
----------
train Loss: 1.5257 Acc: 0.4590
val Loss: 1.0342 Acc: 0.6379

Epoch 2/10
----------
train Loss: 0.9274 Acc: 0.6771
val Loss: 0.7463 Acc: 0.7402

Epoch 3/10
----------
train Loss: 0.6769 Acc: 0.7644
val Loss: 0.6692 Acc: 0.7729

Epoch 4/10
----------
train Loss: 0.5145 Acc: 0.8212
val Loss: 0.6409 Acc: 0.7893

Epoch 5/10
----------
train Loss: 0.3842 Acc: 0.8676
val Loss: 0.6474 Acc: 0.7828

Epoch 6/10
----------
train Loss: 0.2755 Acc: 0.9048
val Loss: 0.5793 Acc: 0.8124

Epoch 7/10
----------
train Loss: 0.1953 Acc: 0.9345
val Loss: 0.6405 Acc: 0.8025

Epoch 8/10
----------
train Loss: 0.0705 Acc: 0.9819
val Loss: 0.5361 Acc: 0.8352

Epoch 9/10
----------
train Loss: 0.0458 Acc: 0.9904
val Loss: 0.5207 Acc: 0.8366

Epoch 10/10
----------
train Loss: 0.0338 Acc: 0.9933
val Loss: 0.5413 Acc: 0.8360

Training complete in 32m 26s
Best val Acc: 0.8366


## Assigment 2:
Load a **pre-trained** (on ImageNet) "*ResNet18*" from the torchvision model zoo and *fine-tune* it for ten epochs


In [11]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

optimizer = optim.SGD([
    {'params': model.fc.parameters(), 'lr': 0.01}
], momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        if epoch == 5:
            print("Unfreezing layer4 for fine-tuning...")
            for param in model.layer4.parameters():
                param.requires_grad = True

            optimizer = optim.SGD([
                {'params': model.fc.parameters(), 'lr': 0.01},
                {'params': model.layer4.parameters(), 'lr': 0.001}
            ], momentum=0.9)

            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = trainloader
            else:
                model.eval()
                dataloader = testloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10
----------
train Loss: 11.1110 Acc: 0.2804
val Loss: 11.8846 Acc: 0.3398

Epoch 2/10
----------
train Loss: 11.6158 Acc: 0.2942
val Loss: 10.7311 Acc: 0.3702

Epoch 3/10
----------
train Loss: 11.6953 Acc: 0.2903
val Loss: 11.7278 Acc: 0.3564

Epoch 4/10
----------
train Loss: 11.6726 Acc: 0.2928
val Loss: 13.4339 Acc: 0.3353

Epoch 5/10
----------
train Loss: 11.5761 Acc: 0.2946
val Loss: 11.2510 Acc: 0.3460

Epoch 6/10
----------
Unfreezing layer4 for fine-tuning...
train Loss: 2.4158 Acc: 0.4942
val Loss: 1.0463 Acc: 0.6673

Epoch 7/10
----------
train Loss: 1.1109 Acc: 0.6486
val Loss: 0.9054 Acc: 0.7116

Epoch 8/10
----------
train Loss: 0.8978 Acc: 0.7064
val Loss: 1.1330 Acc: 0.6734

Epoch 9/10
----------
train Loss: 0.5913 Acc: 0.7976
val Loss: 0.7236 Acc: 0.7607

Epoch 10/10
----------
train Loss: 0.5426 Acc: 0.8126
val Loss: 0.7237 Acc: 0.7638

Training complete in 23m 21s
Best val Acc: 0.7638
